In [1]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


# Deaths

In [2]:
dfDeaths = pd.read_excel('DeathsAgeDK\Deaths.xlsx')
dfDeaths = dfDeaths.transpose()
dfDeaths.columns = dfDeaths.iloc[0]
dfDeaths = dfDeaths.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfDeaths.index,format='%d_%m_%Y')

In [3]:
# dfDeaths.diff().iloc[1:,:-1]

In [4]:
deathVals = dfDeaths.diff().iloc[1:,:-1].values.astype('int64')
deathVals = np.transpose(deathVals)

In [5]:
plt.figure()

g = sns.heatmap(deathVals,annot=True,cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfDeaths.columns[:-1])
g.invert_yaxis()
# np.isnan(np.transpose(deathVals))

plt.title('Dødsfald')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/DeathsHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Indlagte

In [6]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDatesAdm =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [7]:
admVals = dfAdm.diff().iloc[1:,:-1].values.astype('int64')
admVals = np.transpose(admVals)

In [8]:
plt.figure()

g = sns.heatmap(admVals,annot=True,fmt="3",cmap='turbo',xticklabels=curDatesAdm.strftime('%d-%b')[1:],yticklabels=dfAdm.columns[:-1])
g.invert_yaxis()

plt.title('Nyindlæggelser')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AdmittedHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Intensiv 

In [9]:
dfInt = pd.read_excel('Admitted\Intensiv.xlsx')
dfInt = dfInt.transpose()
dfInt.columns = dfInt.iloc[0]
dfInt = dfInt.drop(['Aldersgruppe']) 
curDatesInt =  pd.to_datetime(dfInt.index,format='%d_%m_%Y')

In [10]:
intVals = dfInt.diff().iloc[1:,:-1].values.astype('int64')
intVals = np.transpose(intVals)

In [11]:
plt.figure()

g = sns.heatmap(intVals,annot=True,fmt="3",cmap='turbo',xticklabels=curDatesInt.strftime('%d-%b')[1:],yticklabels=dfInt.columns[:-1])
g.invert_yaxis()

plt.title('Intensiv')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/IntensivHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Across the same age

In [26]:
numCols = len(dfAdm.columns)
dfAdm.columns

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90+', 'I alt'],
      dtype='object', name='Aldersgruppe')

In [48]:

# fig,ax1 = plt.subplots(1,1)
fig,allAxes = plt.subplots(numCols-1,1,sharex=True,figsize=(14,20))

colNames = dfAdm.columns
for curCol in range(0,numCols-1):

    curAx = allAxes[numCols-2-curCol]
    curAx.set_title(colNames[curCol])
    
    curDea = np.diff(dfDeaths.iloc[:,curCol].values)
    curAdm = np.diff(dfAdm.iloc[:,curCol].values)
    curInt = np.diff(dfInt.iloc[:,curCol].values)

    plotDates = curDates[1:]
    lns1 = curAx.plot(plotDates,curAdm,'b:',label='Indlæggelser')
    # curAx.plot(plotDates,curInt,label='Intensiv')
    # curAx.plot(plotDates,curDea,label='Dødsfald')
    curAx2 = curAx.twinx()
    lns2 = curAx.plot(plotDates,curInt,'k--',label='Intensiv')
    lns3 = curAx2.plot(plotDates,curDea,'r',label='Dødsfald')
    
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
        
    curAx.set_ylim(bottom=0)
    curAx2.set_ylim(bottom=0)

# allAxes[4].legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

curAx.set_xlim([curDates[1],curDates[-1]])

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [77]:

# fig,ax1 = plt.subplots(1,1)
fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfAdm.columns
for curCol in range(0,numCols-1):

    # curAx1 = allAxes[(numCols-2-curCol) * 3]
    # curAx2 = allAxes[(numCols-2-curCol) * 3 + 1]
    # curAx3 = allAxes[(numCols-2-curCol) * 3 + 2]
    curAx1 = allAxes[numCols-2-curCol,0]
    curAx2 = allAxes[numCols-2-curCol,1]
    curAx3 = allAxes[numCols-2-curCol,2]
    
    curAx1.set_ylabel(colNames[curCol])
    
    curDea = np.diff(dfDeaths.iloc[:,curCol].values)
    curAdm = np.diff(dfAdm.iloc[:,curCol].values)
    curInt = np.diff(dfInt.iloc[:,curCol].values)

    plotDates = curDates[1:]
    curAx1.plot(plotDates,curAdm,'b:',label='Indlæggelser')
    # curAx.plot(plotDates,curInt,label='Intensiv')
    # curAx.plot(plotDates,curDea,label='Dødsfald')
    curAx2.plot(plotDates,curInt,'k--',label='Intensiv')
    curAx3.plot(plotDates,curDea,'r',label='Dødsfald')
    
        
    curAx1.set_ylim(bottom=0)
    curAx2.set_ylim(bottom=0)
    curAx3.set_ylim(bottom=0)

# allAxes[4].legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

    
curAx1.set_xlim([curDates[0],curDates[-1]])
curAx1.set_xticks(curDates[::3])
curAx1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

allAxes[0,0].set_title('Indlæggelser')
allAxes[0,1].set_title('Intensiv')
allAxes[0,2].set_title('Dødsfald')

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AgeDist_AllData')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:

# fig,ax1 = plt.subplots(1,1)
fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfAdm.columns
for curCol in range(0,numCols-1):

    # curAx1 = allAxes[(numCols-2-curCol) * 3]
    # curAx2 = allAxes[(numCols-2-curCol) * 3 + 1]
    # curAx3 = allAxes[(numCols-2-curCol) * 3 + 2]
    curAx1 = allAxes[numCols-2-curCol,0]
    curAx2 = allAxes[numCols-2-curCol,1]
    curAx3 = allAxes[numCols-2-curCol,2]
    
    curAx1.set_ylabel(colNames[curCol])
    
    curDea = np.diff(dfDeaths.iloc[:,curCol].values)
    curAdm = np.diff(dfAdm.iloc[:,curCol].values)
    curInt = np.diff(dfInt.iloc[:,curCol].values)

    plotDates = curDates[1:]

    plotDates = plotDates[13:]
    curDea = curDea[13:]
    curAdm = curAdm[13:]
    curInt = curInt[13:]


    curAx1.plot(plotDates,curAdm,'b:',label='Indlæggelser')
    # curAx.plot(plotDates,curInt,label='Intensiv')
    # curAx.plot(plotDates,curDea,label='Dødsfald')
    curAx2.plot(plotDates,curInt,'k--',label='Intensiv')
    curAx3.plot(plotDates,curDea,'r',label='Dødsfald')
    
        
    curAx1.set_ylim(bottom=0)
    curAx2.set_ylim(bottom=0)
    curAx3.set_ylim(bottom=0)

# allAxes[4].legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

    
curAx1.set_xlim([plotDates[0],plotDates[-1]])
curAx1.set_xticks(plotDates)
curAx1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

allAxes[0,0].set_title('Indlæggelser')
allAxes[0,1].set_title('Intensiv')
allAxes[0,2].set_title('Dødsfald')

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AgeDist_AllData_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:


# curDea = dfDeaths.iloc[:,curCol].values
# curAdm = dfAdm.iloc[:,curCol].values
# curInt = dfInt.iloc[:,curCol].values

# ax1.plot(curDates,curAdm)
# ax1.plot(curDates,curInt)
# ax1.plot(curDates,curDea)

curDea = np.diff(dfDeaths.iloc[:,curCol].values)
curAdm = np.diff(dfAdm.iloc[:,curCol].values)
curInt = np.diff(dfInt.iloc[:,curCol].values)

plotDates = curDates[1:]
ax1.plot(plotDates,curAdm)
ax1.plot(plotDates,curInt)
ax1.plot(plotDates,curDea)